In [100]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error as mse
from statsmodels.regression.linear_model import OLS
from math import sqrt
import statsmodels

3.1

In [6]:
data = pd.read_csv('covid-data-us-national.txt',index_col='date', parse_dates=['date'])

In [192]:
#split train and test set
train_df = data.loc['2020-03-07':'2020-08-29']
test_df = data.loc['2020-09-05':'2020-09-26']
train = train_df['death_jhu_incidence'].tolist()
test = test_df['death_jhu_incidence'].tolist()

In [193]:
#run arima model for forecasting
forecast_result = []
for i in range(len(test)):
    arima_model = ARIMA(train, order = (2,0,2))
    arima_fit = arima_model.fit()
    forecast = arima_fit.forecast()
    train.append(forecast[0])
    forecast_result.append(forecast[0])
rmse_arima = mse(test,forecast_result,squared=False)
print('rmse of arima: ',rmse_error)

rmse of arima:  1812.0965227401564


C:\Users\Zun Wang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Zun Wang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


3.2

In [189]:
#split train and test data for ols
train_df=data.loc['2020-03-07':'2020-08-29']
train_X = train_df[['mobility','totalTests','covid_cases']]
train_X = statsmodels.tools.tools.add_constant(train_X)
train_Y = train_df['death_jhu_incidence'].tolist()
test_df=data.loc['2020-09-05':'2020-09-26']
test_X_full=test_df[['mobility','totalTests','covid_cases']]
test_X_full=statsmodels.tools.tools.add_constant(test_X_full)
test_Y=test_df['death_jhu_incidence'].tolist()
X=pd.concat([train_X,test_X_full])

ols_result = []
#run ols model, rolling the predicted output into training set
for i in range(len(test_Y)):
    test_X = test_X_full.iloc[i, :]
    ols_model = OLS(train_Y,train_X)
    ols_fit = ols_model.fit()
    forecast = ols_fit.predict(test_X)
    ols_result.append(forecast[0])
    train_Y.append(forecast[0])
    train_X=X.head(train_X.shape[0]+1)

ols_rmse = mse(test_Y,ols_result,squared=False)
print('rmse of ols: ', ols_rmse)

rmse of ols:  1992.0691496008976


3.3

In [205]:
# arima only for training
train_df=data.loc['2020-03-07':'2020-08-29']
train_arima=train_df['death_jhu_incidence'].tolist()
arima_model = ARIMA(train_arima, order=(2,0,2))
arima_fit = arima_model.fit()
forecast_arima = arima_fit.apply(train_arima)
arima_output = forecast_arima.fittedvalues
rmse_arima_train = mse(arima_output,train_arima,squared=False)
print('arima rmse:', rmse_arima)

# OLS only for training
train_Y = train_df['death_jhu_incidence']
train_X = train_df[['mobility','totalTests','covid_cases']]
train_X = statsmodels.tools.tools.add_constant(train_X)

ols_model = OLS(train_Y,train_X)
ols_fit = ols_model.fit()
ols_output = ols_fit.predict(train_X)

rmse_ols_train= mse(ols_output,train_Y,squared=False)
print('ols rmse:' , rmse_ols)

arima rmse: 1812.0965227401564
ols rmse: 2864.671904742172


In [206]:
#EM1
em1 = (arima_output + ols_output) / 2
rmse_em1 = mse(train_Y,em1,squared=False)
print('rmse of EM1:',rmse_em1)
#EM2, applied weight
em2 = ((1/rmse_arima_train)*arima_output+(1/rmse_ols_train)*ols_output)/((1/rmse_arima_train)+(1/rmse_ols_train))
rmse_em2 = mse(train_Y,em2,squared=False)
print('rmse of EM2:',rmse_em2)

rmse of EM1: 1624.75162518402
rmse of EM2: 1507.8709869242582
